In [1]:
import os, sys, glob
import pandas as pd
import json
import argparse
import time
import numpy
from tqdm import tqdm

# Import coffea specific features
from coffea import processor

# SUEP Repo Specific
import SUEP_coffea
from workflows import pandas_utils
from workflows import merger

In [2]:
processor_inst = SUEP_coffea.SUEP_cluster(
    isMC=1,
    era=2018,
    do_syst=1,
    syst_var="",
    sample="QCD_Pt+RunIISummer20UL18",
    weight_syst="",
    flag=False,
    output_location='/home/cms-jovyan/SUEPCoffea_dask'
)

In [12]:
from dask.distributed import Client

client = Client("tls://christos-2epapageorgakis-40cern-2ech.dask.cmsaf-prod.flatiron.hollandhpc.org:8786")
client

Connection method: Direct,
Dashboard: /user/christos.papageorgakis@cern.ch/proxy/8787/status,
Comm: tls://192.168.150.47:8786,Workers: 0
Dashboard: /user/christos.papageorgakis@cern.ch/proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [13]:
import shutil

shutil.make_archive("workflows", "zip", base_dir="workflows")
client.upload_file("workflows.zip")
client.upload_file("SUEP_coffea.py")

{}

In [14]:
result = processor.run_uproot_job(
    fileset="filelist/json_mit/QCD_Pt_slimmed10_xcache.json",
    treename="Events",
    processor_instance=processor_inst,
    executor=processor.dask_executor,
    executor_args={
        "schema": processor.NanoAODSchema,
        "client": client,
    },
    chunksize=5000,
)

2022-08-01 15:58:42,402 - distributed.comm.tcp - WARNING - Closing dangling stream in <TLS  local=tls://192.168.150.47:42230 remote=tls://christos-2epapageorgakis-40cern-2ech.dask.cmsaf-prod.flatiron.hollandhpc.org:8786>
Task was destroyed but it is pending!
task: <Task pending name='Task-1409' coro=<ProgressBar.listen() running at /opt/conda/lib/python3.8/site-packages/distributed/diagnostics/progressbar.py:88> wait_for=<Future cancelled> cb=[IOLoop.add_future.<locals>.<lambda>() at /opt/conda/lib/python3.8/site-packages/tornado/ioloop.py:688]>


Exception: Failed processing file: WorkItem(dataset='QCD_Pt_170to300+RunIISummer20UL18', filename='root://xcache//store/user/paus/nanosu/A01/QCD_Pt_170to300_TuneCP5_13TeV_pythia8+RunIISummer20UL18MiniAODv2-106X_upgrade2018_realistic_v16_L1v1-v1+MINIAODSIM//00EDCD66-B0ED-DD4F-9545-E99D582F1898.root', treename='Events', entrystart=38400, entrystop=43200, fileuuid=b'n\x13\xa5\x06/\x88\x11\xec\x8f\x89\x05\x1b=\x86\xbe\xef', usermeta={})

In [7]:
rslt_pd = {}
for dataset in result:
    rslt_pd[dataset] = {}
    rslt_pd[dataset]['gensum'] = result[dataset]['out_weight'].value
    del result[dataset]['out_weight']
    rslt_pd[dataset]['dataframe'] = pd.DataFrame.from_dict(result[dataset])

In [7]:
for key in rslt_pd:
    processor_inst.gensumweight = rslt_pd[dataset]['gensum']
    pandas_utils.save_dfs(
        processor_inst, 
        dfs=[rslt_pd[key]['dataframe']], 
        df_names=["vars"], 
        fname=f"{key}.hdf5"
    )